# Reverse Video Search with Mixpeek and MongoDB
This notebook demonstrates how to process a video, embed video chunks, and perform a reverse video search using Mixpeek API and MongoDB.

Tutorial: https://learn.mixpeek.com/reverse-video-search/

In [ ]:
!pip install mixpeek pymongo

In [ ]:
# Import necessary libraries
from mixpeek import Mixpeek
from pymongo import MongoClient

In [ ]:
# Initialize Mixpeek client and MongoDB collection

api_key = ''
mongo_uri = ''
db_name = 'demos'
collection_name = 'reverse_video_search'

mixpeek = Mixpeek(api_key)
collection = MongoClient(mongo_uri)[db_name][collection_name]

In [ ]:
# Process video
video_index_url = 'https://mixpeek-public-demo.s3.us-east-2.amazonaws.com/media-analysis/The+Third+Man++Official+Trailer.mp4'

processed_videos = mixpeek.tools.video.process(
    url=video_index_url,
    frame_interval=20,
    resolution=[720, 1280],
    return_base64=True
)

In [ ]:
# Embed video chunks

for index, video in enumerate(processed_videos):
    print(f"embedding video chunk: {index}")
    response = mixpeek.embed.video(
        model_id="mixpeek/vuse-generic-v1",
        input=video['base64_string'],
        input_type="base64"
    )
    obj = {
        "start_time": video['start_time'],
        "end_time": video['end_time'],
        "embedding": response['embedding'],
        "file_url": video_index_url
    }
    collection.insert_one(obj)

In [ ]:
# Perform a reverse video search and display results

response = mixpeek.embed.video(
    model_id="mixpeek/vuse-generic-v1",
    input="https://mixpeek-public-demo.s3.us-east-2.amazonaws.com/media-analysis/video_queries/exiting_sewer.mp4",
    input_type="url"
)

query = [
    {
        "$vectorSearch": {
            "index": "vector_index",
            "path": "embedding",
            "queryVector": response['embedding'],
            "numCandidates": 10,
            "limit": 3
        }
    },
    {
        "$project": {
            "embedding": 0
        }
    }
]

search_results = list(collection.aggregate(query))
search_results